In [11]:
# Copyright (c) Microsoft Corporation.
# Licensed under the MIT License.

"""
Create the .yaml for each experiment
"""
import os
import random




def create_configuration(cfg, cfg_file):
    if 'lossnet' in cfg['algorithm']:
        cfg['save_name'] = "b_{alg}_{dataset}_{net}_{num_lb}_lr{lr}_{steplr}_bsz{batch_size}_{noise_type}_{noise_ratio}_{weakonly}_{esize}_{ebsz}_{metagoal}_{w_f}_{meta_lr}_beta{beta}_{seed}".format(
            alg=cfg['algorithm'],
            dataset=cfg['dataset'],
            net = cfg['net'],
            num_lb=cfg['num_labels'],
            lr=cfg['lr'],
            steplr=cfg['steplr'],
            batch_size = cfg['batch_size'],
            noise_type=cfg['noise_type'],
            noise_ratio =cfg['noise_ratio'],
            weakonly = cfg['weak_only'],
            esize =cfg['eval_set_size'],
            ebsz =cfg['e_batch_size'],
            metagoal = cfg['meta_goal'],
            w_f=cfg['w_f'],
            meta_lr=cfg['meta_lr'],
            beta=cfg['beta'],
            seed=cfg['seed']
        )
    else:
        cfg['save_name'] = "b_{alg}_{dataset}_{net}_{num_lb}_lr{lr}_{steplr}_bsz{batch_size}_{noise_type}_{noise_ratio}_{weakonly}_{esize}_{ebsz}_{metagoal}_{w_f}_beta{beta}_{seed}".format(
            # renew=cfg['renew'],
            alg=cfg['algorithm'],
            dataset=cfg['dataset'],
            net = cfg['net'],
            num_lb=cfg['num_labels'],
            lr=cfg['lr'],
            steplr=cfg['steplr'],
            batch_size = cfg['batch_size'],
            noise_type=cfg['noise_type'],
            noise_ratio =cfg['noise_ratio'],
            weakonly = cfg['weak_only'],
            esize =cfg['eval_set_size'],
            ebsz =cfg['e_batch_size'],
            metagoal = cfg['meta_goal'],
            w_f=cfg['w_f'],
            beta=cfg['beta'],
            seed=cfg['seed']
        )
    # resume
    cfg['resume'] = False
    cfg['load_path'] = '{}/{}/latest_model.pth'.format(cfg['save_dir'], cfg['save_name'])

    alg_file = cfg_file + cfg['algorithm'] + '/'
    if not os.path.exists(alg_file):
        os.mkdir(alg_file)

    print(alg_file + cfg['save_name'] + '.yaml')
    with open(alg_file + cfg['save_name'] + '.yaml', 'w', encoding='utf-8') as w:
        lines = []
        for k, v in cfg.items():
            line = str(k) + ': ' + str(v)
            lines.append(line)
        for line in lines:
            w.writelines(line)
            w.write('\n')




def create_classific_config(alg, seed,
                            dataset, net, num_classes, num_labels, img_size, 
                            port,meta_goal,threshold,noise_type,noise_ratio,meta_lr,beta):
    cfg = {}
    cfg['algorithm'] = alg

    # save config
    cfg['save_dir'] = './saved_models/cifar10_uda'
    cfg['save_name'] = None
    cfg['resume'] = False
    cfg['load_path'] = None
    cfg['overwrite'] = True
    cfg['use_tensorboard'] = True

    # algorithm config
    cfg['epoch'] = 10
    cfg['num_train_iter'] = 120000             #1048576         #2 ** 20
    cfg['num_eval_iter'] = 500
    cfg['num_labels'] = num_labels
    cfg['batch_size'] = 64
    cfg['eval_batch_size'] = 256
    
    if alg == 'fixmatch' or alg == 'fixmatch_lossnet':
        cfg['hard_label'] = True
        cfg['T'] = 0.5
        cfg['p_cutoff'] = 0.95
        cfg['ulb_loss_ratio'] = 1.0
        cfg['uratio'] = 7
    elif alg == 'adamatch':
        cfg['hard_label'] = True
        cfg['T'] = 0.5
        cfg['p_cutoff'] = 0.95
        cfg['ulb_loss_ratio'] = 1.0
        cfg['ema_p'] = 0.999
        cfg['uratio'] = 7
    elif alg == 'flexmatch':
        cfg['hard_label'] = True
        cfg['T'] = 0.5
        cfg['thresh_warmup'] = True
        cfg['p_cutoff'] = 0.95
        cfg['ulb_loss_ratio'] = 1.0
        cfg['uratio'] = 7
    elif alg == 'uda':
        cfg['tsa_schedule'] = 'none'
        cfg['T'] = 0.4
        cfg['p_cutoff'] = 0.8
        cfg['ulb_loss_ratio'] = 1.0
        cfg['uratio'] = 7
    elif alg == 'pseudolabel':
        cfg['p_cutoff'] = 0.95
        cfg['ulb_loss_ratio'] = 1.0
        cfg['uratio'] = 1
        cfg['unsup_warm_up'] = 0.4
    elif alg == 'mixmatch':
        cfg['uratio'] = 1
        cfg['mixup_alpha'] = 0.5
        cfg['T'] = 0.5
        if dataset == 'cifar10':
            cfg['ulb_loss_ratio'] = 75
        elif dataset == 'cifar100':
            cfg['ulb_loss_ratio'] = 150
        else:
            cfg['ulb_loss_ratio'] = 100
        cfg['unsup_warm_up'] = 0.4 # 16000 / 1024 / 1024
    elif alg == 'remixmatch':
        cfg['mixup_alpha'] = 0.75
        cfg['T'] = 0.5
        cfg['kl_loss_ratio'] = 0.5
        cfg['ulb_loss_ratio'] = 1.5
        cfg['rot_loss_ratio'] = 0.5
        cfg['unsup_warm_up'] = 1 / 64
        cfg['uratio'] = 1
    elif alg == 'crmatch':
        cfg['hard_label'] = True
        cfg['p_cutoff'] = 0.95
        cfg['ulb_loss_ratio'] = 1.0
        cfg['uratio'] = 7
    elif alg == 'comatch':
        cfg['hard_label'] = False
        cfg['p_cutoff'] = 0.95
        cfg['contrast_p_cutoff'] = 0.8 
        cfg['contrast_loss_ratio'] = 1.0
        cfg['ulb_loss_ratio'] = 1.0
        cfg['proj_size'] = 64
        cfg['queue_batch'] = 5
        cfg['smoothing_alpha'] = 0.9
        cfg['uratio'] = 7
        cfg['T'] = 0.2
        cfg['da_len'] = 32
        
        if dataset == 'stl10':
            cfg['contrast_loss_ratio'] = 5.0

        if dataset == 'imagenet':
            cfg['p_cutoff'] = 0.6
            cfg['contrast_p_cutoff'] = 0.3
            cfg['contrast_loss_ratio'] = 10.0
            cfg['ulb_loss_ratio'] = 10.0
            cfg['smoothing_alpha'] = 0.9
            cfg['T'] = 0.1
            cfg['proj_size'] = 128

    elif alg == 'simmatch':
        cfg['p_cutoff'] = 0.95
        cfg['in_loss_ratio'] = 1.0
        cfg['ulb_loss_ratio'] = 1.0
        cfg['proj_size'] = 128
        cfg['K'] = 256
        cfg['da_len'] = 32
        cfg['smoothing_alpha'] = 0.9
        cfg['uratio'] = 7
        if dataset in ['cifar10', 'svhn',  'cifar100', 'stl10']:
            cfg['T'] = 0.1
        else:
            cfg['T'] = 0.2
    elif alg == 'meanteacher':
        cfg['uratio'] = 1
        cfg['ulb_loss_ratio'] = 50
        cfg['unsup_warm_up'] = 0.4
    elif alg == 'pimodel':
        cfg['ulb_loss_ratio'] = 10
        cfg['uratio'] = 1
        cfg['unsup_warm_up'] = 0.4
    elif alg == 'dash':
        cfg['gamma'] = 1.27
        cfg['C'] = 1.0001
        cfg['rho_min'] = 0.05
        cfg['num_wu_iter'] = 2048
        cfg['T'] = 0.5
        cfg['p_cutoff'] = 0.95
        cfg['ulb_loss_ratio'] = 1.0
        cfg['uratio'] = 7
    elif alg == 'mpl':
        cfg['tsa_schedule'] = 'none'
        cfg['T'] = 0.7
        cfg['p_cutoff'] = 0.6
        cfg['ulb_loss_ratio'] = 8.0
        cfg['uratio'] = 7
        cfg['teacher_lr'] = 0.03
        cfg['label_smoothing'] = 0.1
        cfg['num_uda_warmup_iter'] = 5000
        cfg['num_stu_wait_iter'] = 3000

    # cfg['img']
    cfg['ema_m'] = 0.999 #0.999
    cfg['crop_ratio'] = 0.875
    cfg['img_size'] = img_size

    # optim config
    cfg['optim'] = 'SGD'
    cfg['lr'] = 0.03
    cfg['momentum'] = 0.9
    cfg['weight_decay'] = 0.0001
    cfg['layer_decay'] = 1.0
    cfg['amp'] = False
    cfg['clip'] = 0.0
    cfg['use_cat'] = True

    # net config
    cfg['net'] = net
    cfg['net_from_name'] = False

    # data config
    cfg['data_dir'] = './data'
    cfg['dataset'] = dataset
    cfg['train_sampler'] = 'RandomSampler'
    cfg['num_classes'] = num_classes
    cfg['num_workers'] = 1

    # basic config
    cfg['seed'] = seed

    # distributed config
    cfg['world_size'] = 1
    cfg['rank'] = 0
    cfg['multiprocessing_distributed'] = False
    cfg['dist_url'] = 'tcp://127.0.0.1:' + str(port)
    cfg['dist_backend'] = 'nccl'
    cfg['gpu'] = None

    # other config
    cfg['overwrite'] = True
    cfg['amp'] = False

   
    
    cfg['lr']= 0.01                       #0.0005                    #0.001     #0.0005 0.01 
    cfg['beta'] = beta
    cfg['num_train_iter'] = 60000 
    cfg['steplr']=True
    cfg['batch_size']= 32
    cfg['noise_type'] = noise_type
    cfg['noise_ratio'] = noise_ratio
    cfg['weak_only']=False
    cfg['eval_set_size']= 7000
    cfg['e_batch_size']= 100
    cfg['meta_goal']= meta_goal
    cfg['w_f']= 'uni'
    cfg['threshold']=threshold
    cfg['use_pretrain']=False
    cfg['renew']= None
    cfg['ema_m'] = 0.999
    cfg['meta_lr'] = meta_lr
    # cfg['relabel']=False
    return cfg



# prepare the configuration for baseline model, use_penalty == False
def exp_classific_cv(label_amount,meta_goal,threshold,noise_type,noise_ratio,meta_lr,beta):
    config_file = r'./config/classic_cv_meta/'
    save_path = r'./saved_models/mnist'

    if not os.path.exists(config_file):
        os.mkdir(config_file)
    if not os.path.exists(save_path):
        os.mkdir(save_path)

    # algs=['fullysupervised']
    algs=['fullysupervised_lossnet']
    # algs = ['flexmatch', 'fixmatch', 'uda', 'pseudolabel', 'fullysupervised', 'remixmatch', 'mixmatch', 'meanteacher',
    #         'pimodel', 'vat', 'dash', 'crmatch', 'comatch', 'simmatch', 'adamatch']
    # datasets = ['cifar100', 'svhn', 'stl10', 'cifar10']
    datasets = ['clothing1m']
    # seeds = [0, 1, 2] 
    seeds = [2,3,8,11,13]


    dist_port = range(10001, 11120, 1)
    count = 0

    for alg in algs:
        for dataset in datasets:
            for seed in seeds:
                
                # change the configuration of each dataset
                if dataset == 'cifar10':
                    # net = 'WideResNet'
                    num_classes = 10
                    num_labels = label_amount
                    weight_decay = 5e-4
                    net = 'wrn_28_10'
                    # weight_decay = 1e-3
                    # net='resnet34'
                    # weight_decay=1e-3
                    img_size = 32

                elif dataset == 'cifar100':
                    # net = 'WideResNet's
                    num_classes = 100
                    num_labels = label_amount
                    weight_decay = 1e-3
                    # depth = 28
                    # widen_factor = 8
                    net = 'wrn_28_10'
                    # net='resnet34'
                    # weight_decay = 5e-4
                    img_size = 32 
                if dataset == 'clothing1m':
                    # net = 'WideResNet'
                    num_classes = 14 
                    num_labels = label_amount
                    # weight_decay = 5e-4
                    # net = 'wrn_28_10'
                    weight_decay = 1e-3
                    net='resnet50'
                    # weight_decay=1e-3
                    img_size = 256
                    
                elif dataset == 'svhn':
                    # net = 'WideResNet'
                    num_classes = 10
                    num_labels = label_amount
                    weight_decay = 5e-4
                    # depth = 28
                    # widen_factor = 2
                    net = 'wrn_28_2'
                    img_size = 32

                elif dataset == 'stl10':
                    # net = 'WideResNetVar'
                    num_classes = 10
                    num_labels = label_amount
                    weight_decay = 5e-4
                    net = 'wrn_var_37_2'
                    img_size = 96
                elif dataset == 'mnist':
                    # net = 'WideResNetVar'
                    num_classes = 10
                    num_labels = label_amount
                    weight_decay = 5e-4
                    net = 'LeNet'
                    img_size = 32

                elif dataset == 'imagenet':
                    if alg not in ['fixmatch', 'flexmatch']:
                        continue
                    net = 'resnet50'
                    num_classes = 1000
                    num_labels = 100000  # 128000
                    weight_decay = 3e-4

                port = dist_port[count]
                
                
                # prepare the configuration file
                cfg = create_classific_config(alg, seed,
                                              dataset, net, num_classes, num_labels, img_size, 
                                              port,meta_goal,threshold,noise_type,noise_ratio,meta_lr,beta)
                count += 1
                create_configuration(cfg, config_file)
                # print(cfg['save_name'])



# if __name__ == '__main__':
#     # if not os.path.exists('./saved_models/classic_cv/'):
#     #     os.mkdir('./saved_models/classic_cv/')
#     if not os.path.exists('./config/classic_cv/'):
#         os.mkdir('./config/classic_cv/')

#     # classific cv
#     label_amount = {'s': [40, 400, 40, 40],
#                     'm': [250, 2500, 250, 250],
#                     'l': [4000, 10000, 1000, 1000]}

    # for i in label_amount:
    #     exp_classific_cv(label_amount=label_amount[i])


In [12]:
    # if not os.path.exists('./saved_models/classic_cv/'):
    #     os.mkdir('./saved_models/classic_cv/')
if not os.path.exists('./config/classic_cv_meta/'):
        os.mkdir('./config/classic_cv_meta/')

label_amount_l=[192000]
# meta_goal_l= ['feat_exp', 'feat_expr', 'feat_expN', 'feat_expNr', 'feat_expno1', 'feat_expno1r', 'feat_expno1N', 'feat_expno1Nr']
# meta_goal_l= ['feat_exp_ws_','feat_exp_ws']
# meta_goal_l=['feat_expno1','feat_expno1N']
meta_goal_l=['cen']
# feat_expno1N
noise_type_l =  ['human']
# noise_type_l =  ['human_worst','human_aggre','human_random']

noise_ratio_l = [0]
# noise_l = ['human_worst_0','asy_0.4']
threshold_l=[0]
meta_lr_l=[0.001] #0.0001 0.00001 1e-05
beta_l=[0]

In [13]:
for label_amount in label_amount_l:
    for meta_goal in meta_goal_l:
        for threshold in threshold_l:
            for noise_type in noise_type_l:
                for noise_ratio in noise_ratio_l:
                    for meta_lr in meta_lr_l:
                        for beta in beta_l:
                            exp_classific_cv(label_amount,meta_goal,threshold,noise_type,noise_ratio,meta_lr,beta)    

./config/classic_cv_meta/fullysupervised_lossnet/b_fullysupervised_lossnet_clothing1m_resnet50_192000_lr0.01_True_bsz32_human_0_False_7000_100_cen_uni_0.001_beta0_2.yaml
./config/classic_cv_meta/fullysupervised_lossnet/b_fullysupervised_lossnet_clothing1m_resnet50_192000_lr0.01_True_bsz32_human_0_False_7000_100_cen_uni_0.001_beta0_3.yaml
./config/classic_cv_meta/fullysupervised_lossnet/b_fullysupervised_lossnet_clothing1m_resnet50_192000_lr0.01_True_bsz32_human_0_False_7000_100_cen_uni_0.001_beta0_8.yaml
./config/classic_cv_meta/fullysupervised_lossnet/b_fullysupervised_lossnet_clothing1m_resnet50_192000_lr0.01_True_bsz32_human_0_False_7000_100_cen_uni_0.001_beta0_11.yaml
./config/classic_cv_meta/fullysupervised_lossnet/b_fullysupervised_lossnet_clothing1m_resnet50_192000_lr0.01_True_bsz32_human_0_False_7000_100_cen_uni_0.001_beta0_13.yaml
